# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [94]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
# nltk.download('vader_lexicon')
from nltk.tokenize import word_tokenize
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
### nltk.download('averaged_perceptron_tagger')
### nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords


## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
# ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

## 0.1 Load and clean text data

In [95]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [96]:
# Filter to get the press release with ID "17-1204"
pharma_release = doj[doj['id'] == "17-1204"]

# Extract the contents of the press release
pharma_contents = pharma_release['contents'].iloc[0]

# Convert the contents to a single string
pharma = str(pharma_contents)

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [97]:
# Part A
preprocessed_pharma1 = re.sub(r'\W+', ' ', pharma)


In [98]:
# Part B
tokens = word_tokenize(preprocessed_pharma)
tagged_words = nltk.pos_tag(tokens)

In [99]:
# Part C
from collections import Counter

# Extract adjectives from tagged words
adjectives = [word for word, tag in tagged_words if tag.startswith('JJ')]

# Count occurrences of adjectives
adj_counts = Counter(adjectives)

# Convert the Counter to a DataFrame
adj_df = pd.DataFrame(adj_counts.items(), columns=['Adjective', 'Count'])

# Sort the DataFrame by count in descending order
adj_df = adj_df.sort_values(by='Count', ascending=False)
print(adj_df.head(5))


     Adjective  Count
10      former      8
27      opioid      5
0   nationwide      4
31   addictive      3
9        other      3


## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [100]:
# Part A
# Load the English language model
nlp = spacy.load('en_core_web_sm')

# Process the original pharma press release
doc = nlp(pharma)

# Extract named entities
named_entities = [(entity.text, entity.label_) for entity in doc.ents]


In [101]:
# Part B
print("Unique Named Entities with Tag 'LAW':", set(entity.text for entity in doc.ents if entity.label_ == 'LAW'))

Unique Named Entities with Tag 'LAW': {'the Controlled Substances Act', 'RICO'}


C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

The main aim of the Racketeer Influenced and Corrupt Organizations Act (RICO) was to combat organized crime like the Mafia. But over time, some have tried to stretch its scope to create a federal law against deceptive advertising by pharmaceutical companies, seeking hefty compensation and legal costs. The Ninth Circuit's recent decision has further supported these attempts by allowing RICO claims against pharmaceutical firms for their alleged mislabeling and promotional practices concerning prescription drugs.

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [176]:


# Extract named entities with the label "DATE"
date_entities = [entity.text for entity in doc.ents if entity.label_ == 'DATE']

# Filter date entities containing the word "year" or "years" using regex
sentence_lengths = [date for date in date_entities if re.search(r'\b(?:year|years)\b', date, flags=re.IGNORECASE)]

# Print the extracted possible sentence lengths the CEO is facing
print("Possible sentence lengths the CEO is facing:")
for length in sentence_lengths:
    print(length)

Possible sentence lengths the CEO is facing:
last year
20 years
three years
five years
three years


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [103]:
# Define a regular expression pattern to match sentences containing the identified year lengths
pattern = r'(?:^|(?<=[.!?]))[^.!?]*\b(?:year|years)\b[^.!?]*(?:[.!?]|$)'

# Find all sentences matching the pattern in the pharma press release
matching_sentences = re.findall(pattern, pharma)

# Print the original parts of the press releases where year lengths are mentioned
print("Original Parts of the Press Release where Year Lengths are Mentioned:")
for sentence in matching_sentences:
    print(sentence)

# Describe the potential sentence and probation lengths the CEO may face
if matching_sentences:
    # Extract the maximum year length mentioned
    max_year_length = max([int(re.search(r'\b\d+\b', date).group()) for date in sentence_lengths if re.search(r'\b\d+\b', date)])

    # Describe the maximum sentence and probation lengths
    sentence_description = f"The CEO may be facing a maximum sentence of {max_year_length} years in prison."
    probation_description = f"If convicted after this indictment, the CEO may face a maximum supervised release (probation) of {max_year_length} years."

    print("\nDescription:")
    print(sentence_description)
    print(probation_description)
else:
    print("\nNo year lengths mentioned in the press release.")

Original Parts of the Press Release where Year Lengths are Mentioned:
 "More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids.
The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.
  The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.

Description:
The CEO may be facing a maximum sentence of 20 years in prison.
If convicted after this indictment, the CEO may face a maximum supervised release (probation) of 20 years.


## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [104]:
# Define the list of topics to filter
topics_to_filter = ['Civil Rights', 'Hate Crimes', 'Project Safe Childhood']

# Filter press releases based on the specified topics
doj_subset = doj[doj['topics_clean'].isin(topics_to_filter)]

# Check the number of rows in the subset
print("Number of rows in doj_subset:", len(doj_subset))


Number of rows in doj_subset: 717


B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [105]:
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize the SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

def process_press_release(press_release):
    # Remove named entities using regular expressions
    processed_release = re.sub(r'\b[A-Z][a-z]*\b', '', press_release)
    
    # Score the sentiment of the processed string
    sentiment_scores = sia.polarity_scores(processed_release)
    
    # Return the sentiment dictionary
    return sentiment_scores


In [177]:
# Apply the function to each press release in doj_subset
sentiment_scores_list = [process_press_release(press_release) for press_release in doj_subset['contents']]
sentiment_scores_list
# Print the sentiment scores
#for i, scores in enumerate(sentiment_scores_list):
   # print(f"Press Release {i+1} Sentiment Scores:", scores)

[{'neg': 0.207, 'neu': 0.739, 'pos': 0.054, 'compound': -0.992},
 {'neg': 0.125, 'neu': 0.799, 'pos': 0.075, 'compound': -0.8843},
 {'neg': 0.094, 'neu': 0.827, 'pos': 0.078, 'compound': -0.738},
 {'neg': 0.13, 'neu': 0.782, 'pos': 0.088, 'compound': -0.8981},
 {'neg': 0.187, 'neu': 0.767, 'pos': 0.046, 'compound': -0.9864},
 {'neg': 0.148, 'neu': 0.79, 'pos': 0.062, 'compound': -0.9833},
 {'neg': 0.156, 'neu': 0.763, 'pos': 0.081, 'compound': -0.9541},
 {'neg': 0.089, 'neu': 0.843, 'pos': 0.068, 'compound': -0.7128},
 {'neg': 0.106, 'neu': 0.83, 'pos': 0.064, 'compound': -0.9022},
 {'neg': 0.164, 'neu': 0.774, 'pos': 0.062, 'compound': -0.9733},
 {'neg': 0.227, 'neu': 0.735, 'pos': 0.038, 'compound': -0.9973},
 {'neg': 0.099, 'neu': 0.833, 'pos': 0.067, 'compound': -0.8417},
 {'neg': 0.089, 'neu': 0.839, 'pos': 0.071, 'compound': -0.617},
 {'neg': 0.341, 'neu': 0.628, 'pos': 0.031, 'compound': -0.9957},
 {'neg': 0.184, 'neu': 0.756, 'pos': 0.06, 'compound': -0.9907},
 {'neg': 0.129, '

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [107]:
# Apply the process_press_release function to each press release in doj_subset
sentiment_scores_list = [process_press_release(press_release) for press_release in doj_subset['contents']]

# Create a DataFrame with sentiment scores
doj_subset_wscore = doj_subset.copy()
doj_subset_wscore['neg'] = [scores['neg'] for scores in sentiment_scores_list]
doj_subset_wscore['neu'] = [scores['neu'] for scores in sentiment_scores_list]
doj_subset_wscore['pos'] = [scores['pos'] for scores in sentiment_scores_list]
doj_subset_wscore['compound'] = [scores['compound'] for scores in sentiment_scores_list]

# Sort doj_subset_wscore from highest neg to lowest neg score
doj_subset_wscore = doj_subset_wscore.sort_values(by='neg', ascending=False)

# Print the top id, contents, and neg columns of the two most negative press releases
print("Top 2 Most Negative Press Releases:")
print(doj_subset_wscore[['id', 'contents', 'neg']].head(2))


Top 2 Most Negative Press Releases:
           id  \
329    14-248   
11593  16-718   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [108]:
# Group by topics_clean and calculate the mean compound sentiment score for each group
mean_compound_scores = doj_subset_wscore.groupby('topics_clean')['compound'].agg('mean')

# Print the mean compound sentiment score for each topic
print("Mean Compound Sentiment Score for Each Topic:")
print(mean_compound_scores)


Mean Compound Sentiment Score for Each Topic:
topics_clean
Civil Rights             -0.109418
Hate Crimes              -0.936392
Project Safe Childhood   -0.707904
Name: compound, dtype: float64


The variation in mean compound sentiment scores across different topics could be attributed to the varying nature and intensity of the events and issues associated with each topic. For instance, topics like Hate Crimes might elicit more negative sentiment due to the severity and emotional impact of such crimes, resulting in lower mean compound scores compared to topics like Civil Rights, where sentiments may be more mixed or nuanced. Similarly, Project Safe Childhood, while focusing on child safety, may involve sensitive and distressing subject matter, contributing to a lower mean compound score.

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [113]:
# Initialize Snowball stemmer and stopwords
snowball_stemmer = SnowballStemmer(language='english')
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

def preprocess_text(raw_text):
    # Convert the text to lowercase
    raw_text = raw_text.lower()
    
    # Tokenize the text into words
    words = nltk.word_tokenize(raw_text)
    
    # Remove stopwords, custom stopwords, punctuation, and non-alpha words, and retain words longer than 3 characters
    filtered_words = [word for word in words if word not in set(stopwords.words('english') + list(string.punctuation) + custom_doj_stopwords) and len(word) >= 4 and word.isalpha()]
    
    # Stem the words using Snowball stemmer
    stemmed_words = [snowball_stemmer.stem(word) for word in filtered_words]
    
    # Join the stemmed words back into a single string
    preprocessed_text = ' '.join(stemmed_words)
    
    return preprocessed_text

In [114]:
# Apply the preprocess_text function to each row in the contents column and create a new column processed_text
doj_subset_wscore['processed_text'] = doj_subset_wscore['contents'].apply(preprocess_text)


In [116]:
# Filter the DataFrame for the specified press releases
selected_press_releases = doj_subset_wscore[doj_subset_wscore['id'].isin(['16-718', '16-217'])]

# Print the id, contents, and processed_text columns for the selected press releases
print(selected_press_releases[['id', 'contents', 'processed_text']])

           id  \
11593  16-718   
6727   16-217   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [130]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase=True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis=1)
    return dtm_dense_named_withid

In [154]:
# Create the document-term matrix using the provided create_dtm function
dtm = create_dtm(doj_subset_wscore['processed_text'], doj_subset_wscore[['id', 'compound', 'topics_clean']])

In [155]:
def get_topwords(subset_data):
    # Concatenate all preprocessed text
    all_text = ' '.join(subset_data['processed_text'])

    # Create a document-term matrix for the concatenated text
    dtm_subset = create_dtm([all_text], pd.DataFrame())

    # Get the top 10 words
    top_10_words = dtm_subset.iloc[0, 3:].sort_values(ascending=False)[:10]
    
    return top_10_words


In [156]:
# Step B: Top 10 words for press releases with compound sentiment in the top 5%
top_5_percent = doj_subset_wscore[doj_subset_wscore['compound'] >= threshold]
topwords_top_5_percent = get_topwords(top_5_percent)
print("Top 10 words for press releases with compound sentiment in the top 5%:")
print(topwords_top_5_percent)

Top 10 words for press releases with compound sentiment in the top 5%:
agreement     173.0
enforc        133.0
state         118.0
disabl        110.0
ensur         108.0
communiti      94.0
polic          93.0
settlement     86.0
general        86.0
student        82.0
Name: 0, dtype: float64


In [134]:
# Part B
# Calculate the threshold for the top 5% compound sentiment score
threshold = doj_subset_wscore['compound'].quantile(0.95)

# Filter press releases with compound sentiment in the top 5%
top_5_percent = doj_subset_wscore[doj_subset_wscore['compound'] >= threshold]

# Concatenate all preprocessed text from press releases in the top 5%
top_text = ' '.join(top_5_percent['processed_text'])

# Create a document-term matrix for the press releases in the top 5%
dtm_top_5_percent = create_dtm([top_text], pd.DataFrame())

# Get the top 10 words
top_10_words = dtm_top_5_percent.iloc[0, 3:].sort_values(ascending=False)[:10]

# Print the top 10 words
print("Top 10 words for press releases with compound sentiment in the top 5%:")
print(top_10_words)


Top 10 words for press releases with compound sentiment in the top 5%:
agreement     173.0
enforc        133.0
state         118.0
disabl        110.0
ensur         108.0
communiti      94.0
polic          93.0
settlement     86.0
general        86.0
student        82.0
Name: 0, dtype: float64


In [157]:
# Step C: Top 10 words for press releases with compound sentiment in the bottom 5%
bottom_5_percent = doj_subset_wscore[doj_subset_wscore['compound'] <= threshold_bottom]
topwords_bottom_5_percent = get_topwords(bottom_5_percent)
print("\nTop 10 words for press releases with compound sentiment in the bottom 5%:")
print(topwords_bottom_5_percent)


Top 10 words for press releases with compound sentiment in the bottom 5%:
assault     198.0
victim      172.0
crime       160.0
hate        127.0
offic       120.0
defend      117.0
sentenc     111.0
charg       101.0
anderson     93.0
guilti       92.0
Name: 0, dtype: float64


In [143]:
# Part C
# Calculate the threshold for the bottom 5% compound sentiment score
threshold_bottom = doj_subset_wscore['compound'].quantile(0.05)

# Filter press releases with compound sentiment in the bottom 5%
bottom_5_percent = doj_subset_wscore[doj_subset_wscore['compound'] <= threshold_bottom]

# Concatenate all preprocessed text from press releases in the bottom 5%
bottom_text = ' '.join(bottom_5_percent['processed_text'])

# Create a document-term matrix for the concatenated text
dtm_bottom_5_percent = create_dtm([bottom_text], pd.DataFrame())

# Get the top 10 words
top_10_words_bottom = dtm_bottom_5_percent.iloc[0, 3:].sort_values(ascending=False)[:10]

# Print the top 10 words
print("Top 10 words for press releases with compound sentiment in the bottom 5%:")
print(top_10_words_bottom)


Top 10 words for press releases with compound sentiment in the bottom 5%:
assault     198.0
victim      172.0
crime       160.0
hate        127.0
offic       120.0
defend      117.0
sentenc     111.0
charg       101.0
anderson     93.0
guilti       92.0
Name: 0, dtype: float64


In [158]:
# Step D: Top 10 words for press releases in each of the three topics_clean
for topic, subset_data in doj_subset_wscore.groupby('topics_clean'):
    print(f"\nTop 10 words for press releases in '{topic}':")
    topwords_topic = get_topwords(subset_data)
    print(topwords_topic)


Top 10 words for press releases in 'Civil Rights':
offic        627.0
hous         620.0
discrimin    541.0
enforc       531.0
disabl       509.0
said         497.0
feder        475.0
violat       470.0
state        443.0
general      408.0
Name: 0, dtype: float64

Top 10 words for press releases in 'Hate Crimes':
victim      590.0
crime       533.0
prosecut    476.0
hate        472.0
defend      459.0
sentenc     455.0
charg       452.0
guilti      430.0
feder       426.0
said        424.0
Name: 0, dtype: float64

Top 10 words for press releases in 'Project Safe Childhood':
child          1018.0
exploit         698.0
sexual          570.0
safe            476.0
project         472.0
childhood       472.0
pornographi     447.0
children        416.0
crimin          404.0
prosecut        374.0
Name: 0, dtype: float64


In [141]:
# Part D
def get_top_words(subset_data):
    # Concatenate all preprocessed text from the subset
    subset_text = ' '.join(subset_data['processed_text'])
    
    # Create a document-term matrix for the concatenated text
    dtm_subset = create_dtm([subset_text], pd.DataFrame())
    
    # Get the top 10 words
    top_10_words_subset = dtm_subset.iloc[0, 3:].sort_values(ascending=False)[:10]
    
    return top_10_words_subset

# Define a function to print the top 10 words for a given topic
def print_top_10_words_for_topic(topic):
    # Filter press releases for the given topic
    topic_press_releases = doj_subset_wscore[doj_subset_wscore['topics_clean'] == topic]
    
    # Get the top words for the topic using the get_top_words function
    top_words_topic = get_top_words(topic_press_releases)
    
    # Print the top 10 words for the topic
    print(f"Top 10 words for press releases in the topic '{topic}':")
    print(top_words_topic)
    print("\n")

# Iterate over each topic and print the top 10 words
for topic in ['Civil Rights', 'Hate Crimes', 'Project Safe Childhood']:
    print_top_10_words_for_topic(topic)



Top 10 words for press releases in the topic 'Civil Rights':
offic        627.0
hous         620.0
discrimin    541.0
enforc       531.0
disabl       509.0
said         497.0
feder        475.0
violat       470.0
state        443.0
general      408.0
Name: 0, dtype: float64


Top 10 words for press releases in the topic 'Hate Crimes':
victim      590.0
crime       533.0
prosecut    476.0
hate        472.0
defend      459.0
sentenc     455.0
charg       452.0
guilti      430.0
feder       426.0
said        424.0
Name: 0, dtype: float64


Top 10 words for press releases in the topic 'Project Safe Childhood':
child          1018.0
exploit         698.0
sexual          570.0
safe            476.0
project         472.0
childhood       472.0
pornographi     447.0
children        416.0
crimin          404.0
prosecut        374.0
Name: 0, dtype: float64




## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [175]:
## your code here to get doc-level topic probabilities 


In [ ]:
## your code here to add those topic probabilities to the dataframe

In [ ]:
## your code here to summarize the topic proportions for each of the topics_clean 

# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [166]:
# Part A
def create_bigram_onedoc(text):
    # Split the text into words
    words = text.split()
    # Use zip to iterate over pairs of consecutive words
    bigrams = ['{}_{}'.format(words[i], words[i+1]) for i in range(len(words)-1)]
    # Join the bigrams with spaces
    return ' '.join(bigrams)

doj_subset_wscore['processed_text_bigram'] = doj_subset_wscore['processed_text'].apply(create_bigram_onedoc)


# text_example = "depart reach settlem"
# bigram_example = create_bigram_onedoc(text_example)
# print(bigram_example)


In [167]:
# Part B
# Filter the DataFrame to get the row with the specified ID
press_release_16_217 = doj_subset_wscore[doj_subset_wscore['id'] == '16-217']

# Extract the required columns
id_16_217 = press_release_16_217['id'].values[0]
processed_text_16_217 = press_release_16_217['processed_text'].values[0]
processed_text_bigram_16_217 = press_release_16_217['processed_text_bigram'].values[0]

# Print the id, processed_text, and processed_text_bigram columns
print("ID:", id_16_217)
print("Processed Text:", processed_text_16_217)
print("Processed Text Bigram:", processed_text_bigram_16_217)


ID: 16-217
Processed Text: reach comprehens settlement agreement citi miami miami polic resolv shoot offic announc princip deputi general vanita gupta head wifredo ferrer southern florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim shoot effect quick investig shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi general gupta agreement help strengthen relationship communiti serv improv account offic fire weapon unlaw provid communiti particip enforc today agreem

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

In [170]:
# Part C
# Create a document-term matrix using the processed_text_bigrams column
dtm_bigram = create_dtm(doj_subset_wscore['processed_text_bigram'], doj_subset_wscore[['id', 'topics_clean', 'compound']])

In [171]:
# Part D
# Print the dimensions of the dtm matrix from question 2.2
print("Dimensions of dtm matrix from question 2.2:", dtm.shape)

# Print the dimensions of the dtm_bigram matrix
print("Dimensions of dtm_bigram matrix:", dtm_bigram.shape)

Dimensions of dtm matrix from question 2.2: (717, 6760)
Dimensions of dtm_bigram matrix: (717, 71336)


The bigram matrix has more dimensions than the unigram matrix because it considers pairs of consecutive words (bigrams) as features, whereas the unigram matrix considers single words (unigrams) as features. Therefore, the bigram matrix captures more detailed information about the relationships between words in the text, leading to a higher number of dimensions.

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [173]:
# Define a function to get the top 10 bigrams for each topic
def get_top_bigrams(dtm, topic_column, topic_name):
    # Subset the dtm to the specific topic
    topic_dtm = dtm[dtm[topic_column] == topic_name]
    # Sum the frequencies of each bigram
    bigram_counts = topic_dtm.iloc[:, 3:].sum(axis=0)
    # Get the top 10 bigrams
    top_bigrams = bigram_counts.sort_values(ascending=False).head(10)
    return top_bigrams

# Print the top 10 bigrams for each of the three topics_clean
for topic in ['Civil Rights', 'Hate Crimes', 'Project Safe Childhood']:
    print("Top 10 bigrams for", topic, ":")
    top_bigrams = get_top_bigrams(dtm_bigram, 'topics_clean', topic)
    print(top_bigrams)
    print()

Top 10 bigrams for Civil Rights :
fair_hous         227.0
deputi_general    221.0
princip_deputi    221.0
vanita_gupta      202.0
gupta_head        200.0
general_vanita    199.0
said_princip      186.0
unit_state        152.0
nation_origin     141.0
consent_decre     127.0
dtype: float64

Top 10 bigrams for Hate Crimes :
hate_crime       368.0
plead_guilti     275.0
year_prison      160.0
special_agent    117.0
thoma_perez      111.0
grand_juri       101.0
perez_general     95.0
said_thoma        91.0
unit_state        87.0
act_general       85.0
dtype: float64

Top 10 bigrams for Project Safe Childhood :
project_safe         472.0
safe_childhood       472.0
child_pornographi    445.0
child_exploit        279.0
sexual_exploit       223.0
exploit_children     199.0
plead_guilti         197.0
exploit_obscen       175.0
child_sexual         175.0
obscen_section       174.0
dtype: float64



# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [ ]:
# your code here 